# Нулевые гипотезы, статистические тесты

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

np.random.seed(7)

В проверках нулевых гипотез формулируют гипотезу $H_0$ о данных $\mathcal{D}$. Выбирают статистический тест $T$ - случайную величину с известным распределением $P_{T}(x | H_0)$ в предположении $H_0$. Считают реализацию величины $T$ в данных - тестовую статистику $x_{0}$ [[TestStat](https://en.wikipedia.org/wiki/Test_statistic)]. Вероятность получить фактическое или более экстремальное значение тестовой статистики называют $p$-значением $p = P_{T}(x \ge x_{0} | H_0)$ [[PVal](https://en.wikipedia.org/wiki/P-value), [TailedTests](https://en.wikipedia.org/wiki/One-_and_two-tailed_tests)]. Если вероятность «достаточно мала», гипотезу $H_0$ «отвергают», если нет - «оставляют».

<center>
<img src="../figs/null_hypothesis.png" alt="null_hypothesis" width="800"/>
<em> В проверках нулевых гипотез формулируют гипотезу $H_0$ и выбирают статистический тест $T$ - случайную величину с известным распределением $P_{T}(x | H_0)$ в предположении $H_0$. Считают реализацию величины $T$ в данных - тестовую статистику $x_{0}$. Вероятность получить фактическое или более экстремальное значение тестовой статистики называют $p$-значением $p = P_{T}(x \ge x_{0} | H_0)$. Если вероятность «достаточно мала», гипотезу $H_0$ «отвергают», если нет - «оставляют». </em>
</center>

Основная проблема метода - решение принимается по $p$-значению $p = P_{T}(x \ge x_{0} | H_0)$, тогда как для оценки гипотезы нужна вероятность $P(H_0 | x_0)$. По соотношению Байеса $P(H_0 | x_0) \propto P_{T}(x = x_{0} | H_0) P(H_0)$. Т.е. для выбора гипотезы нужно посчитать вероятности получить данные в рамках конкурирующих гипотез и сравнить друг с другом с учетом априорных вероятностей.  

В А/Б-тестах нужно выбрать группу с большим значением целевой метрики. Распространенные применения проверок нулевых гипотез к А/Б-тестам включают $t$-тест для средних, $\chi^2$-тест пропорций и $U$-критерий Манна-Уитни. Для них $p$-значение численно близко вероятности метрик одной группы больше другой, хотя отличается по определению.

## Т-тест

Средние могут сравнивать $t$-тестами [[TTest](https://en.wikipedia.org/wiki/Student%27s_t-test)]. Пусть есть выборки размера $N_A, N_B$ из двух случайных величин $A, B$. По выборочным средним $\mu_A, \mu_B$ и дисперсиям $s_A^2, s_B^2$ оценивают среднее и дисперсию разности $\mu_{\Delta} = \mu_B - \mu_A$, $s^2_{\Delta} = s_A^2/N_A + s_B^2/N_B$. Считают отношение $x_0 = \mu_{\Delta}/s_{\Delta}$. Предполагают, что средние в группах одинаковы. Тогда для отношения $\mu_{\Delta}/s_{\Delta}$ ожидают $t$-распределение. При достаточно большом количестве данных оно близко стандартному нормальному $\text{Norm}(0, 1)$ [[WelchT](https://en.wikipedia.org/wiki/Welch%27s_t-test)]. Вычисляют вероятность получить фактическое $x_0$ или более экстремальное отношение - $p$-значение $P_{\mu_{\Delta}/s_{\Delta}}(x > x_0 | \mu_A = \mu_B)$. Если оно "достаточно мало", считают средние в группах неравными.

$$
x_0 = \frac{\mu_{\Delta}}{s_{\Delta}},
\quad
\mu_{\Delta} = \mu_B - \mu_A,
\quad
s^2_{\Delta} = \frac{s_A^2}{N_A} + \frac{s_B^2}{N_B}
\\
P_{\mu_{\Delta}/s_{\Delta}}(x | \mu_A = \mu_B) \approx \text{Norm}(x; 0, 1)
\\
p = P_{\mu_{\Delta}/s_{\Delta}}(x > x_0 | \mu_A = \mu_B)
$$

В А/Б-тесте нужно выбрать группу с большим средним. Поэтому вместо $p$-значения $P_{\mu_{\Delta}/s_{\Delta}}(x > x_0 | \mu_A = \mu_B)$ интересна вероятность среднего $B$ больше $A$ при условии собранных данных $P(\mu_B > \mu_A | x_0 )$. Эту вероятность можно оценить байесовским моделированием. В пренебрежении априорными значениями приближенно $\mu_B - \mu_A \sim \text{Norm}(\mu_{\Delta}, s^2_{\Delta})$. Поэтому $P(\mu_B > \mu_A | x_0 ) \approx P(\text{Norm}(x < 0 | x_0, 1))$. В общем случае связь $p$-значения с этой вероятностью не очевидна. Для $t$-тестов по симметрии нормального распределения $P(\text{Norm}(x > x_0 | 0, 1)) =  P(\text{Norm}(x < 0 | x_0, 1))$. Поэтому $p$-значение одностороннего $t$-теста близко вероятности среднего одной группы больше другой.

$$
\begin{split}
P_{\mu_{\Delta}/s_{\Delta}}(x > x_0 | \mu_A = \mu_B)
& = P(\text{Norm}(x > x_0 | 0, 1)) 
\\
& =  P(\text{Norm}(x < 0 | x_0, 1)) 
\approx P(\mu_B > \mu_A | x_0 )
\end{split}
$$

На графике ниже 2 нормальных распределения. Одно с центром в точке 0, другое в точке $x_0 = 2$. Вероятность 
$P(x > x_0 | \mu_A = \mu_B)$ p-значение закрашено темным, $P(\mu_B > \mu_A | x_0 )$ закрашено светлым. По свойствам нормального распределения площади этих областей совпадают. 

In [ ]:
mud = 2
s = 1

xaxis_min = -7
xaxis_max = 7
x = np.linspace(xaxis_min, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=0, scale=s), 
                         line_color='black', opacity=0.8, name=f'$Norm(0, 1)$'))
fig.add_trace(go.Scatter(x=[mud, mud], y=[0, max(stats.norm.pdf(x, loc=0, scale=s))*1.1], 
                         line_color='black', 
                         mode='lines+text', text=['', '$x_0$'], textposition="top center",
                         line_dash='dash', showlegend=False))
fig.add_trace(go.Scatter(x=x[x>mud], y=stats.norm.pdf(x[x>mud], loc=0, scale=s), 
                         line_color='black', opacity=0.8, name=f'$P(x > x_0 | \Delta \mu=0)$', fill="tozeroy", fillcolor="rgba(0, 0, 0, 0.7)"))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mud, scale=s), 
                         line_color='black', opacity=0.3, name=f'$Norm(x_0, 1)$'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(stats.norm.pdf(x, loc=0, scale=s))*1.1], 
                         line_color='black', 
                         mode='lines+text', text=['', '0'], textposition="top center", 
                         line_dash='dash', showlegend=False))
fig.add_trace(go.Scatter(x=x[x<0], y=stats.norm.pdf(x[x<0], loc=mud, scale=s), 
                         line_color="rgba(128, 128, 128, 0.3)", name=f'$P(x < 0 | \Delta \mu / s_\Delta = x_0)$', fill="tozeroy", fillcolor="rgba(128, 128, 128, 0.3)"))
fig.update_layout(title='P-значение и вероятность среднего одной группы больше другой',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[xaxis_min, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()

Т.е. $p$-значение одностороннего $t$-теста близко байесовской оценке вероятности, что среднее одной группы больше другой. Ниже заданы два распределения Бернулли с разными конверсиями. Из них делается выборка. По выброрке строится байесовская оценка вероятности $P(p_B > p_A)$ и вычисляется $t$-тест. $T$-тест задается односторонний с разными дисперсиями групп (параметры `equal_var=False, alternative='greater'`). Видно, что p-значение близко байесовской оценке вероятности. При этом стоит помнить, что они не совпадают в точности - у них разные определения, также p-значение игнорирует априорные вероятности гипотез. 

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)

def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

pA = 0.1
pB = pA * 1.05

exactA = stats.bernoulli(pA)
exactB = stats.bernoulli(pB)

N = 30000
sampA = exactA.rvs(size=N)
sampB = exactB.rvs(size=N)

post_dist_A = posterior_dist_binom(ns=np.sum(sampA), ntotal=N)
post_dist_B = posterior_dist_binom(ns=np.sum(sampB), ntotal=N)
pb_gt_pa = prob_pb_gt_pa(post_dist_A, post_dist_B)

# p_diff = sampB.mean() - sampA.mean()
# s_diff = np.sqrt(sampA.std()**2 / N + sampB.std()**2 / N)
# diff = stats.norm(loc=p_diff, scale=s_diff)
# pb_gt_pa = diff.cdf(0)

t_stat, p_value = stats.ttest_ind(sampA, sampB, equal_var=False, alternative='greater')

print(f'P(pb > pa): {pb_gt_pa}')
print(f'p-value P(x>x0 | pa=pb): {p_value}')

Интерпретацию $p$-значения $t$-теста можно проверить по количеству правильно угаданных вариантов с большим значением конверсии в серии экспериментов. 

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best_bayes', 'p-val'])

p = 0.1
nexps = 300
cmp['A'] = [p] * nexps
cmp['B'] = p * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))
cmp['best_exact'] = cmp.apply(lambda r: 'B' if r['B'] > r['A'] else 'A', axis=1)

n_samp_max = 3_000_000
n_samp_step = 10_000
prob_stop = 0.95

for i in range(nexps):
    pA = cmp.at[i, 'A']
    pB = cmp.at[i, 'B']
    exact_dist_A = stats.bernoulli(p=pA)
    exact_dist_B = stats.bernoulli(p=pB)
    n_samp_total = 0
    ns_A = 0
    ns_B = 0
    while n_samp_total < n_samp_max:
        dA = exact_dist_A.rvs(n_samp_step)
        dB = exact_dist_B.rvs(n_samp_step)
        n_samp_total += n_samp_step
        ns_A = ns_A + np.sum(dA)
        ns_B = ns_B + np.sum(dB)
        T_A = np.zeros(n_samp_total)
        T_A[:ns_A] = 1
        T_B = np.zeros(n_samp_total)
        T_B[:ns_B] = 1
        t_stat, p_value = stats.ttest_ind(T_A, T_B, equal_var=False, alternative='greater')
        best_gr = 'B' if p_value >= prob_stop else 'A' if (1 - p_value) >= prob_stop else None
        if best_gr:
            post_dist_A = posterior_dist_binom(ns=ns_A, ntotal=n_samp_total)
            post_dist_B = posterior_dist_binom(ns=ns_B, ntotal=n_samp_total)
            pb_gt_pa_bayes = prob_pb_gt_pa(post_dist_A, post_dist_B)
            cmp.at[i, 'A_exp'] = ns_A / n_samp_total
            cmp.at[i, 'B_exp'] = ns_B / n_samp_total
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best_bayes'] = max(pb_gt_pa_bayes, 1 - pb_gt_pa_bayes)
            cmp.at[i, 'p-val'] = max(p_value, 1 - p_value)
            break
    print(f'done {i}: nsamp {n_samp_total}, best_gr {best_gr}, Bayes P(b>a) {pb_gt_pa_bayes:.4f}, T-test p-val {p_value:.4f}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(30))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

## Тест $\chi^2$ 

Хи-квадрат - сумма квадратов нормальных случайных величин.

$$
X \sim \text{Binomial}(n,p), \quad O = x, \quad E = np
\\
\chi^2
=
\frac{(x - np)^2}{np}
+
\frac{((n - x) - n(1-p))^2}{n(1-p)}
=
\frac{(x - np)^2}{np(1-p)}
\\
Z = \frac{x - np}{\sqrt{np(1-p)}}, \quad \chi^2 = Z^2, \quad \chi^2 \sim \chi^2_1
$$

Для конверсий 2 групп эквивалентен расчету разности средних.  
В итоге $p$-значение близко вероятности одной группы больше другой.  

## U-критерий Манна-Уитни

$U$- статистика - попарное сравнение элементов в выборках

$$
U_A = \sum_{i=1}^{n_A} \sum_{j=1}^{n_B} I(X_i > Y_j) 
      + \frac{1}{2} \sum_{i=1}^{n_A} \sum_{j=1}^{n_B} I(X_i = Y_j)
\\
\text{where } I(\text{condition}) =
\begin{cases} 
1 & \text{if condition is true} \\ 
0 & \text{if condition is false} 
\end{cases}
$$

Можно записать через сумму рангов

$$
U_A = R_A - \frac{n_A (n_A + 1)}{2}
\\
\text{where } 
R_A = \sum_{i=1}^{n_A} \text{rank}(X_i)
\text{ is the sum of ranks of group A in the combined dataset.}
$$

$n_A (n_A + 1)/2$ - минимальный ранг если все элементы A меньше B, считается как арифметическая прогрессия.
Если наибольший элемент A больше $n$ элементов B, то $U_A = n$ и $R_A = n_A (n_A + 1)/2 + n$.

Интерпретация $U$-статистики: $U_A / n_A n_B$ - вероятность элемента выборки A больше B. $U_A$ - число элементов A больше B, $n_A n_B$ - общее количество пар.

$$
\frac{U_A}{n_A n_B} = P(X > Y) + \frac{1}{2} P(X = Y)
\\
\text{where } 
X \sim \text{an observation from group A}, \quad
Y \sim \text{an observation from group B}.
$$

$U/n_A n_B$ интерпретируется как вероятность элемента A больше B.  
Сравниваются не средние, а все распределение.  
Вероятность $U/n_A n_B$ может не доходить до 1.   
Будет набираться дольше.

Нулевая гипотеза - выборки A и B из одинакового распределения. 

$$
Z = \frac{U - E[U]}{\sqrt{\text{Var}(U)}} \sim N(0,1)
\\
\text{where } 
E[U] = \frac{n_A n_B}{2}, \quad
\text{Var}(U) = \frac{n_A n_B (n_A + n_B + 1)}{12}
\\
n_A, n_B \gg 10
$$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import mannwhitneyu

mu1, sigma1 = 0, 1
mu2, sigma2 = 0.005, 1
exactA = stats.norm(loc=mu1, scale=sigma1)
exactB = stats.norm(loc=mu2, scale=sigma2)

p_b_gt_a_norm = stats.norm.cdf(0, loc=mu2-mu1, scale=np.sqrt(sigma1**2 + sigma2**2))

nstep = 1000
nmax = 100000
u_norm_values = []
p_values = []
n = []
sampA = np.array([])
sampB = np.array([])

for ntotal in range(nstep, nmax, nstep):
    dA = exactA.rvs(nstep)
    dB = exactB.rvs(nstep)
    sampA = np.concatenate([sampA, dA])
    sampB = np.concatenate([sampB, dB])
    U, p = mannwhitneyu(sampA, sampB, alternative='less')
    n.append(ntotal)
    u_norm_values.append(U / (ntotal * ntotal))
    p_values.append(p)

    

x = np.linspace(-10, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, y=exactA.pdf(x),
    mode='lines', name='A'
))
fig.add_trace(go.Scatter(
    x=x, y=exactB.pdf(x),
    mode='lines', name='B'
))
fig.update_layout(
    title="A, B",
    template="plotly_white",
    yaxis_range=[0,1.1]
)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=n, y=u_norm_values,
    mode='lines+markers',
    name='Normalized U (U / n^2)'
))
fig.add_trace(go.Scatter(x=[n[0], n[-1]], y=[p_b_gt_a_norm, p_b_gt_a_norm], name='P(B>A) exact'))
fig.add_trace(go.Scatter(
    x=n, y=p_values,
    mode='lines+markers',
    name='p-values'
))
fig.update_layout(
    title="Mann-Whitney U Statistic vs Sample Size",
    xaxis_title="Sample size (n_A = n_B)",
    yaxis_title="Normalized U",
    legend_title="Metric",
    template="plotly_white",
    yaxis_range=[0,1.1]
)
fig.show()

Аналитическое значение P(X > Y): интеграл по всем x от плотности вероятности f_X(x) * функцию распределения F_Y(x) * dx (конкретное x * вероятность выпадения в X * вероятность выпадения такого или меньшего значения в Y).

Для нормальных A, B аналитически: $P(B>A) = F_{B-A}(0) = \Phi(0, \mu_B - \mu_A, s_A^2 + s_B^2)$

Важна формулировка гипотезы. В качестве нулевой гипотезы $H_0$ часто предполагают равенство групп (нулевой эффект). В А/Б-тестах нужно проверить не равенство групп, а выбрать группу с большим значением целевой метрики. Поэтому вместо гипотез вида $H_0: p_A = p_B$ нужны $H: p_A > p_B$.

В общем случае использовать $p$-значение для оценки гипотез некорректно. 


В распространных применениях проверок нулевых гипотез к А/Б-тестам $p$-значение оказывается численно близко вероятности метрик одной группы больше другой. Ниже рассмотрены $t$-теста, $\chi^2$-теста для двух пропорций и $U$-критерия Манна-Уитни.

В проверках нулевых гипотез используют статистическую значимость $\alpha$ и мощность $1-\beta$. Они определяют вероятность P(Выбор !H0 \cap H0) и P(Выбор H0 \cap !H0). Их также называют ошибками первого и второго рода. Задание $\alpha$ и $\beta$ не достаточно для задания вероятности корректного определения гипотезы. Она также зависит от качества гипотезы $P(H_0)$. Также полезно подумать о дальнейших действиях: в А/Б - тесте оставляют вариант со значимой разницей. Т.е. нужно оставлять P(выбор !H0 \cap H0) + P(выбор !H0 \cap !H0).

In [ ]:
def approx_ttest_conv_pval(sa, na, sb, nb):
    pa = sa / na
    pb = sb / nb
    stderr_a = np.sqrt(pa * (1 - pa) / na)
    stderr_b = np.sqrt(pb * (1 - pb) / nb)
    diff = pb - pa
    diff_stderr = np.sqrt(stderr_a**2 + stderr_b**2)
    pval = stats.norm.cdf(0, diff, diff_stderr)
    return pval

Основная проблема метода - решение принимается по $p$-значению $p = P_{T}(x \ge x_{0} | H_0)$, тогда как для оценки гипотезы нужна вероятность $P(H_0 | x_0)$. Ее можно получить по соотношению Байеса $P(H_0 | x_0) \propto P_{T}(x = x_{0} | H_0) P(H_0)$. Т.е. посчитать вероятности получить данные в рамках конкурирующих гипотез и сравнить друг с другом с учетом априорных вероятностей. В общем случае $p$-значение не позволяет делать корректных выводов о рассматриваемой гипотезе.

Если приходится иметь дело с проверками нулевых гипотез, важно обращать внимание на формулировку гипотезы. В качестве $H_0$ часто предполагают равенство групп (нулевой эффект). В А/Б-тестах нужно проверить не равенство групп, а выбрать группу с большим значением целевой метрики. Поэтому вместо гипотез вида $H_0: p_A = p_B$ нужны $H: p_A > p_B$.

Для анализа А/Б-тестов используют метод проверки статистических гипотез [[StTest](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing)]. Чаще всего в таком подходе предполагают, что между вариантами нет разницы, после чего смотрят, насколько такое предположение объясняет экспериментальные данные. Если вероятность получить данные мала, считают, что предположение можно отвергнуть и между группами есть значимая разница.  

Есть экспериментальные данные $\mathcal{D}$ и гипотеза $H_0$ о данных. Выбирают «статистический тест» $T$ - случайную величину с известным распределением $P_{T}(x | H_0)$ в предположении $H_0$. По данным считают «тестовую статистику»  $x_{0}$ [[TestStat](https://en.wikipedia.org/wiki/Test_statistic)]. Вероятность получить «фактическое или более экстремальное» значение тестовой статистики называют «$p$-значением» [[PVal](https://en.wikipedia.org/wiki/P-value)]. В зависимости от контекста $p = P_{T}(x \ge x_{0} | H_0)$, $p = P_{T}(x \le x_{0} | H_0)$ или $p = P_{T}(|x - x_{0}| \ge 0 | H_0)$ [[TailedTests](https://en.wikipedia.org/wiki/One-_and_two-tailed_tests)]. Если вероятность «достаточно мала», гипотезу $H_0$ «отвергают», если нет - «не отвергают».